In [ ]:
pip install geopandas shapely requests tqdm

In [ ]:
pip install rasterio scipy

In [ ]:
pip install fiona

In [ ]:
import geopandas as gpd
import os
import glob

import json

fichier_parcelles = "parcelles_filtrées.geojson"
fichier_dalles = "LIDAR.shp"  # couche IGN avec URLs


colonne_url = "url"  # nom de la colonne avec les .copc.laz
sortie_html = "Liens_LIDAR.html"

# === CHARGEMENT DES DONNÉES ===
print("Chargement des couches...")
parcelles = gpd.read_file(fichier_parcelles)
dalles = gpd.read_file(fichier_dalles)

# Harmonisation des projections
if parcelles.crs != dalles.crs:
    dalles = dalles.to_crs(parcelles.crs)

# === SÉLECTION DES DALLAGES ===
print("Recherche des dalles intersectant les parcelles...")
dalles_utiles = gpd.sjoin(dalles, parcelles, how="inner", predicate="intersects")
urls_uniques = dalles_utiles[colonne_url].dropna().unique()

# === CRÉATION DU FICHIER HTML ===
print(f"{len(urls_uniques)} URLs trouvées. Génération du fichier HTML...")

with open(sortie_html, "w", encoding="utf-8") as f:
    f.write("<html><body>\n")
    f.write("<h2>Liens des dalles LiDAR intersectant les parcelles</h2>\n")
    f.write("<ul>\n")
    for url in sorted(urls_uniques):
        name = os.path.basename(url)
        f.write(f'<li><a href="{url}">{name}</a></li>\n')
    f.write("</ul>\n</body></html>")

print(f"✅ Fichier HTML généré ici : {sortie_html}")
